In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import base64

In [ ]:
news = {
    'politics' : {
        'topic' : ['federal','nsw','victoria','queensland','act'],
        'api' : ['federal','nsw','victoria','queensland','act'],
    },
    'business' : {
        'topic' : ['economy','market','company','finance','small-business','consumer','workplace'],
        'api' : ['the-economy','markets','companies','banking-and-finance','small-business','consumer-affairs','workplace'],
    },
    'sport' : {
        'topic' : ['soccer','cricket','racing','motorsport','netball','cycling','tennis','basketball',
                   'golf','football','athletics','swimming','boxing','sailing'],
        'api' : ['soccer','cricket','racing','motorsport','netball','cycling','tennis','basketball',
                   'golf','nfl','athletics','swimming','boxing','sailing'],
    },
    'entertainment' : {
        'topic' : ['movies','tv-radio','music','celebrity','book','comedy','dance','musical','opera','theatre','art'],
        'api' : ['movies','tv-and-radio','music','celebrity','books','comedy','dance','musicals','opera','theatre','art-and-design'],
    },
    'lifestyle' : {
        'topic' : ['life','health','gender','fashion','beauty'],
        'api' : ['life-and-relationships','health-and-wellness','gender','fashion','beauty'],
    },
    'money' : {
        'topic' : ['retirement','investing','banking','borrowing','saving','tax','planning','insurance'],
        'api' : ['super-and-retirement','investing','banking','borrowing','saving','tax','planning-and-budgeting','insurance'],
    },
    'environment' : {
        'topic' : ['conservation','climate','sustainability','weather'],
        'api' : ['conservation','climate-change','sustainability','weather'],
    },
    'technology' : {
        'topic' : ['game','tech','phone','gadget'],
        'api' : ['video-games','home-tech','phones','gadgets'],
    },
}

js처리가 되어 있어서 since 뒷 부분을 base64 to ascii로 decode해서 알아냄.


마지막 기사의 id를 since 값으로 넘겨주어야함("since" : " **Asset:"id"** " 굵은글씨부분을 base64로 encode해야함)

In [ ]:
# get article url
def get_url(url):
    urls = []
    asset = ''
    url_smh = 'https://www.smh.com.au'
    
    for num in range(0,10000):
        id = 'Asset:"'+ asset +'"'
        url_req = url + base64.b64encode(id.encode()).decode('ascii') + '"}'
        
        response = requests.get(url_req).text
        json_response = json.loads(response)['data']['assetsConnection']

        for result in json_response['assets']:
            urls.append(url_smh + result['urls']['canonical']['path'])
            asset = result['id']
    
    return urls

필요없는 부분을 제외시키기 위해서 div태그의 class명이 noPrint가 있는 부분 위까지로 슬라이싱한다.

In [ ]:
# get article content
def get_content(url):
    content = ''
    response = requests.get(url).text
    soup = BeautifulSoup(response, 'html.parser')
    crawl = soup.select('div[id="content"] article > section')
    
    for html in crawl:
        if html.select('div[class="noPrint"]'):
            html = str(html).split('"noPrint"')[0]
            html = BeautifulSoup(html, 'html.parser')
            
        for line in html.select('p'):
            content += line.text.strip()
    
    return content

한번에 최대 49개까지 요청 가능

In [ ]:
url = 'https://api.ffx.io/graphql?query=query%20CategoryIndexMoreQuery(%20%24brand%3A%20String!%20%24count%3A%20Int!%20%24path%3A%20String!%20%24since%3A%20ID%20%24types%3A%20%5BString!%5D!%20)%20%7B%20assetsConnection%3A%20assetsConnectionByNavigationPath(brand%3A%20%24brand%2C%20path%3A%20%24path%2C%20count%3A%20%24count%2C%20sinceID%3A%20%24since%2C%20types%3A%20%24types)%20%7B%20...AssetsConnectionFragment_showMoreData%20%7D%20%7D%20fragment%20AssetsConnectionFragment_showMoreData%20on%20AssetsConnection%20%7B%20assets%20%7B%20...AssetFragmentFragment_assetDataWithTag%20id%20%7D%20pageInfo%20%7B%20endCursor%20hasNextPage%20%7D%20%7D%20fragment%20AssetFragmentFragment_assetDataWithTag%20on%20Asset%20%7B%20...AssetFragmentFragment_assetData%20tags%20%7B%20primaryTag%20%7B%20...AssetFragment_tagFragment%20%7D%20%7D%20%7D%20fragment%20AssetFragmentFragment_assetData%20on%20Asset%20%7B%20id%20asset%20%7B%20about%20byline%20duration%20headlines%20%7B%20headline%20%7D%20live%20totalImages%20%7D%20label%20urls%20%7B%20canonical%20%7B%20path%20brand%20%7D%20published%20%7B%20brisbanetimes%20%7B%20path%20%7D%20canberratimes%20%7B%20path%20%7D%20smh%20%7B%20path%20%7D%20theage%20%7B%20path%20%7D%20watoday%20%7B%20path%20%7D%20%7D%20%7D%20featuredImages%20%7B%20landscape16x9%20%7B%20...ImageFragment%20%7D%20landscape3x2%20%7B%20...ImageFragment%20%7D%20portrait2x3%20%7B%20...ImageFragment%20%7D%20square1x1%20%7B%20...ImageFragment%20%7D%20%7D%20assetType%20dates%20%7B%20modified%20published%20%7D%20sponsor%20%7B%20name%20%7D%20%7D%20fragment%20AssetFragment_tagFragment%20on%20AssetTagDetails%20%7B%20displayName%20urls%20%7B%20published%20%7B%20brisbanetimes%20%7B%20path%20%7D%20canberratimes%20%7B%20path%20%7D%20smh%20%7B%20path%20%7D%20theage%20%7B%20path%20%7D%20watoday%20%7B%20path%20%7D%20%7D%20%7D%20%7D%20fragment%20ImageFragment%20on%20Image%20%7B%20data%20%7B%20animated%20aspect%20autocrop%20cropWidth%20id%20mimeType%20offsetX%20offsetY%20zoom%20%7D%20%7D%20\
&variables={"brand":"smh","count":49,"types":["article","bespoke","featureArticle","liveArticle","video"],"path":"/'
url_ls = []
file = open('smh_news.csv','w',encoding = 'utf-8')

In [ ]:
for category in news:
    print(category)
    for topic,api in zip(news[category]['topic'],news[category]['api']):
        url_api = url + category + '/' + topic  + '","since":"'
        url_ls = get_url(url_api)
        
        for article in url_ls:
            content = get_content(article)
            if content is not '':
                file.write('"{0}","{1}","{2}"\n'.format(category,topic,content))
                
file.close()

In [ ]:
f = open('./smh_news.csv','r',encoding='utf-8')
line = f.read()
print(line)
f.close()